In [52]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta 




In [53]:
#data with feature engineering
fields = ['Date', 'Open','High','Low','Close','Volume','RSI','MACD','Ichimoku_Span_A','Ichimoku_Span_B','ATR',
         'BB_High_Indicator','BB_Low_Indicator']
MSFT_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/MSFT.csv", usecols=fields)
AAPL_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AAPL.csv", usecols=fields)
AMZN_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AMZN.csv", usecols=fields)



In [54]:
MSFT_df_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/MSFT_monthly.csv", index_col=0)
AAPL_df_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/AAPL_monthly.csv", index_col=0)
AMZN_df_monthly =pd.read_csv("/Users/fahad/project_repo/data/external/AMZN_monthly.csv", index_col=0)



In [55]:
MSFT_df_monthly.columns = ["Open","High","Low","Close","Volume"]

#drop the last row of each dataframe so the entries match
#MSFT_df_monthly.drop(MSFT_df_monthly.tail(1).index,inplace=True)

AAPL_df_monthly.columns =["Open","High","Low","Close","Volume"]
#AAPL_df_monthly.drop(AAPL_df_monthly.tail(1).index,inplace=True)

AMZN_df_monthly.columns = ["Open","High","Low","Close","Volume"]
#AMZN_df_monthly.drop(AMZN_df_monthly.tail(1).index,inplace=True)

In [56]:
#reversed monthly Dataframe 
reversed_df_MSFT = MSFT_df_monthly.iloc[::-1]
reversed_df_AAPL= AAPL_df_monthly.iloc[::-1]
reversed_df_AMZN = AMZN_df_monthly.iloc[::-1]

In [57]:
def labelling_multiclass(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    dataframe = dataframe.apply(pd.to_numeric)
    #introduce the absolute price change as a variable
    dataframe['ATR'] = ta.volatility.average_true_range(dataframe.High, dataframe.Low, dataframe.Close, n=14, 
                                                        fillna=True)
    
    #calculate the percentage change of ATR between months 
    dataframe['ATR_%_change'] = abs(dataframe['ATR'].pct_change())
    
    #calculate the average ATR pct change
    average_ATR_pct_change = dataframe['ATR_%_change'].mean()
    
    
    
    empty_dict = dict()
    prev_close =0
    counter = 0
    
    #print (dataframe.head(5))
    
    for index,cols in dataframe.iterrows():
        
        counter +=1
        
        current_close = cols['Close']
        ATR_price_change = cols['ATR_%_change']
        
        if (float(prev_close) <= float(current_close)) and (float(ATR_price_change) <= float(average_ATR_pct_change)):
            
            #signals an increase in price between last month and next month
            
            empty_dict[index] = 1
        # 2 represents an increase that was strong greater than the average ATR   
        elif (float(prev_close) <= float(current_close)) and (float(ATR_price_change) >= float(average_ATR_pct_change)):
            empty_dict[index] = 2
        
        elif (float(prev_close) >= float(current_close)and (float(ATR_price_change) <= float(average_ATR_pct_change))):
            empty_dict[index]=0
            
        #3 represents a decrease in the price that had a larger ATR    
        elif (float(prev_close) >= float(current_close)and (float(ATR_price_change) >= float(average_ATR_pct_change))):
            empty_dict[index] = 3
        
        else:
            empty_dict[index]= 0
            

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [58]:
#ADD LABELS TO THE REVERESED DATAFRAMES USING THE LABELLING FUNCTION 
MSFT_labels = labelling_multiclass(reversed_df_MSFT)
AAPL_labels = labelling_multiclass(reversed_df_AAPL)
AMZN_labels = labelling_multiclass(reversed_df_AMZN)

In [59]:
#Add the labels to the reversed DataFrames
reversed_df_MSFT['Labels'] = MSFT_labels.values()
reversed_df_AAPL['Labels'] = AAPL_labels.values()
reversed_df_AMZN['Labels']= AMZN_labels.values()



/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [60]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            empty_dict[month_key]= temp_df.values.flatten()
            count +=1
        else:
            pass
       
    return empty_dict
    
#get dictionary of the period end dates with the 30days previous features minus date as a flattened array of 360 entries
#reshape 30,12 will give the original dataframe config
MSFT_30days_flattened= get_ndays_monthly_flattened(reversed_df_MSFT,MSFT_df,30)
AAPL_30days_flattened = get_ndays_monthly_flattened(reversed_df_AAPL,AAPL_df,30)
AMZN_30days_flattened = get_ndays_monthly_flattened(reversed_df_AMZN,AMZN_df,30)


In [66]:
def get_ndays_monthly(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the REVERESED dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily. So for example will have 
    a dataframe of the month ending of february 2019-02-28:30days of January"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
        #print (end)
#         print ("\n")
        
        month_key = dataframe_period_end.index[count+1]
        #print ("month key:%s" % month_key)
        last_index = dataframe_period_end.index[-1]
        #print ("last index:%s" % last_index)
        
        if (month_key != last_index):
#         print ("index at position %s is :%s" % (count,dataframe_period_end.index[count]))     
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("month key: %s" % month_key)
#             print ("\n")
#             print (temp_df.head(5))
        
            empty_dict[month_key] = temp_df
        
            count +=1
        else:
            pass
       
    return empty_dict




In [72]:
MSFT_30days= get_ndays_monthly(reversed_df_MSFT,MSFT_df,30)
AAPL_30days = get_ndays_monthly(reversed_df_AAPL,AAPL_df,30)
AMZN_30days = get_ndays_monthly(reversed_df_AMZN,AMZN_df,30)

    

In [73]:
#convert the nonflattened dictionaries to dataframes 
df_MSFT_30 = pd.DataFrame.from_dict(MSFT_30days).T
df_AAPL_30 = pd.DataFrame.from_dict(AAPL_30days).T
df_AMZN_30 = pd.DataFrame.from_dict(AMZN_30days).T

In [62]:
#assign the labels to pandas dataframe objects and give label title
MSFT_labels_df = pd.Series(MSFT_labels).to_frame("Label")
AAPL_labels_df= pd.Series(AAPL_labels).to_frame("Label")
AMZN_labels_df= pd.Series(AMZN_labels).to_frame("Label")

,0
1998-03-31,Date Open High Low...
1998-04-30,Date Open High Low...
1998-05-29,Date Open High Low...
1998-06-30,Date Open High Low...
1998-07-31,Date Open High Low...
1998-08-31,Date Open High Low...
1998-09-30,Date Open High Low...
1998-10-30,Date Open High Low...
1998-11-30,Date Open High Low...
1998-12-31,Date Open High Low...


In [63]:
#convert the flattened dictionaries to dataframes 
flattened_df_MSFT = pd.DataFrame.from_dict(MSFT_30days_flattened).T
flattened_df_AAPL = pd.DataFrame.from_dict(AAPL_30days_flattened).T
flattened_df_AMZN = pd.DataFrame.from_dict(AMZN_30days_flattened).T

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
1998-03-31,0.429339,0.425285,0.432434,0.426907,0.020233,0.627285,0.373432,0.500535,0.519246,0.231693,...,0.710965,0.703805,0.005767,0.226505,0.019309,0.457683,0.712130,0.453455,0.0,0.0
1998-04-30,0.444974,0.443069,0.449092,0.441714,0.005799,0.403578,0.305363,0.475177,0.588950,0.276414,...,0.864060,0.844894,0.012340,0.954846,0.966738,0.509452,0.514381,0.640977,0.0,0.0
1998-05-29,0.457188,0.460549,0.464632,0.457249,0.015414,0.691593,0.397587,0.531022,0.598253,0.255372,...,0.404028,0.404151,0.008533,0.233407,0.259587,0.494930,0.558306,0.266415,0.0,0.0
1998-06-30,0.430866,0.429412,0.436288,0.431822,0.007196,0.440966,0.357870,0.644377,0.636887,0.290104,...,0.468175,0.464470,0.011004,0.510333,0.361165,0.499703,0.595590,0.298256,0.0,0.0
1998-07-31,0.551484,0.558813,0.561785,0.560046,0.022155,0.358593,0.250476,0.596634,0.595164,0.371578,...,0.432807,0.430305,0.028320,0.486961,0.394788,0.617644,0.629785,0.248232,0.0,0.0
1998-08-31,0.590815,0.594805,0.595972,0.596456,0.025066,0.742265,0.363924,0.579208,0.595625,0.418341,...,0.469294,0.461436,0.010805,0.175438,0.183561,0.619970,0.629359,0.436546,0.0,1.0
1998-09-30,0.568279,0.570527,0.558677,0.546756,0.025059,0.586977,0.305362,0.559590,0.666430,0.563441,...,0.633640,0.618059,0.013599,0.647949,0.373647,0.596837,0.595164,0.462133,0.0,0.0
1998-10-30,0.590021,0.597111,0.599826,0.592997,0.014853,0.749450,0.477445,0.667748,0.790036,0.540372,...,0.574963,0.583166,0.024149,0.619997,0.394155,0.590247,0.650735,0.460496,0.0,0.0
1998-11-30,0.551118,0.557660,0.562966,0.555070,0.016410,0.363004,0.362740,0.807038,0.934522,0.387918,...,0.555942,0.547485,0.014265,0.461979,0.382239,0.648793,0.768056,0.522893,0.0,0.0
1998-12-31,0.671308,0.682872,0.680259,0.685175,0.005320,0.481884,0.325906,0.917761,0.984234,0.610075,...,0.554015,0.545603,0.023310,0.477506,0.372636,0.735941,0.888573,0.389915,0.0,0.0


In [31]:
#merge the dataframes on the index 'Date', and ensure that only the dates present in the flattened dataframe are carried over

MSFT_flat_df = flattened_df_MSFT.merge(MSFT_labels_df, left_index=True, right_index=True)
AAPL_flat_df = flattened_df_AAPL.merge(AAPL_labels_df, left_index=True, right_index=True)
AMZN_flat_df = flattened_df_AMZN.merge(AMZN_labels_df, left_index=True, right_index=True)

In [43]:
#split the data into training and test and unfortunately we miss a couple months in the middle for some strange reason?
MSFT_flat_df_98_15 = MSFT_flat_df[:204]
MSFT_flat_df_15_19 = MSFT_flat_df[206:]

In [48]:
#split the data approximately 80% to 20%
AAPL_flat_df_98_15 = AAPL_flat_df[:204]
AAPL_flat_df_15_19 = AAPL_flat_df[206:]

In [50]:
#split the data approximately into %80 to %20 
AMZN_flat_df_98_15 = AMZN_flat_df[:204]
AMZN_flat_df_15_19 = AMZN_flat_df[206:]

In [ ]:
export_98_15 = MSFT_flat_df_98_15.to_csv("multiclass_MSFT_98_15.csv")
export_15_19 = MSFT_flat_df_15_19.to_csv("multiclass_MSFT_15_19.csv")
export_98_15 = AAPL_flat_df_98_15.to_csv("multiclass_AAPL_98_15.csv")
export_15_19 = AAPL_flat_df_15_19.to_csv("multiclass_AAPL_15_19.csv")
export_98_15 = AMZN_flat_df_98_15.to_csv("multiclass_AMZN_98_15.csv")
export_15_19 = AMZN_fl.to_csv("multiclass_AMZN_15_19.csv")
